In [ ]:
#|default_exp monkeydispatch

# Monkey Dispatch

Plum's a powerful typedispatching library that brings multipledispatch to python.

Sadly, it doesn't allow for monkeydispatching, combining multiple dispatch with monkeypatch.

In [ ]:
#|export
import typing
from plum.function import Function
from plum.dispatcher import Dispatcher
from plum.util import get_class

In [ ]:
from plum import dispatch

## Intro to Plum

Plum brings a global `dispatch` that lets you perform multiple dispatch wherever you want.

For example, Plum lets you dispatch methods in classes:

In [ ]:
class A:
    @dispatch
    def encodes(self,x:int): return x*2
    @dispatch
    def encodes(self,x:str): return "h_"+x

a = A()
a.encodes(5), a.encodes('a')  # Expected

(10, 'h_a')

In [ ]:
dispatch

Dispatcher(warn_redefinition=False, functions={}, classes={'__main__.A': {'encodes': <multiple-dispatch function A.encodes (with 2 registered and 0 pending method(s))>}})

So far so good.  But there are two issues.

### Issue 1: monkeydispatching is not allowed

**Note:** Under the hood dispatch is defined as an instance of plum's own Dispatcher class. We'll use that in this nb to make our example self-contained.

```python
# file: plum.dispatcher.py
dispatch = Dispatcher()
```

In [ ]:
dispatch = Dispatcher()
class A:
    @dispatch
    def encodes(self,x:int): return x*2

@dispatch
def encodes(self:A,x:str): return "h_"+x
    
len(A.encodes.methods)  # 🚨 Expected 2 b/c our second patch

1

In [ ]:
dispatch

Dispatcher(warn_redefinition=False, functions={'encodes': <multiple-dispatch function encodes (with 0 registered and 1 pending method(s))>}, classes={'__main__.A': {'encodes': <multiple-dispatch function A.encodes (with 1 registered and 0 pending method(s))>}})

See: one time encodes is stored inside class `__main__.A` and on time as a function.

### Issue 2: class redefinitions don't clear dispatch cache.

In [ ]:
dispatch = Dispatcher()
class A:
    @dispatch
    def encodes(self,x:int): return x*2
class A:
    @dispatch
    def encodes(self,x:str): return "h_"+x
    
len(A.encodes.methods)  # 🚨 Expected 1 b/c class redefinition

2

In [ ]:
dispatch

Dispatcher(warn_redefinition=False, functions={}, classes={'__main__.A': {'encodes': <multiple-dispatch function A.encodes (with 2 registered and 0 pending method(s))>}})

## Attempt at solution

Below I try to address this issue by extending Dispatcher.

In [ ]:
#|export
def _has_self_arg(f) -> bool:
    try: return f.__code__.co_varnames[0] == 'self'
    except (AttributeError, IndexError): return False

In [ ]:
#|export
class MonkeyDispatcher(Dispatcher):    
    def __call__(self, f):
        nm = f.__name__
        if not _has_self_arg(f): return super().__call__(f)
        if cls:= typing.get_type_hints(f).get('self'):
            if cm := getattr(cls,nm,None):
                if type(cm) is Function: cm.dispatch(f)
                else: setattr(cls,nm, Function(cm).dispatch(cm).dispatch(f))
            else: setattr(cls,nm,Function(f).dispatch(f))
            return cls
        else:
            gc = get_class(f)  # __main__.A
            nc, nm = f.__qualname__.split(".") #A, encodes
            if nc in globals(): 
                _ = globals().pop(nc)  # being redefined anyway so can be deleted
                self.classes.get(gc,{}).pop(nm,None)
            return super().__call__(f)    

**Note:** we do some potentially brittle things with `globals()` I haven't really used it before so I'd love feedback on this.

### The original features still work

In [ ]:
dispatch = MonkeyDispatcher()

In [ ]:
class A:
    @dispatch
    def encodes(self,x:int): return x*2
    @dispatch
    def encodes(self,x:str): return "h_"+x

a = A()
assert a.encodes(5) == 10
assert a.encodes('a') == 'h_a'  # Expected

### Benefit 1: Monkeydispatching now works

In [ ]:
class A:
    @dispatch
    def encodes(self,x:int): return x*2

@dispatch
def encodes(self:A,x:str): return "h_"+x
    
assert len(A.encodes.methods) == 2  # 🎉 Expected 2 b/c our second patch

Even if the original method was not decorated

In [ ]:
class A:
    def encodes(self,x:int): return x*2

@dispatch
def encodes(self:A,x:str): return "h_"+x
    
assert len(A.encodes.methods) == 2 # 🎉 Expected 2 b/c our second patch

### Benefit 2: Redefinitions of the class now clear the cache

In [ ]:
class A:
    @dispatch
    def encodes(self,x:int): return x*2
    @dispatch
    def encodes(self,x:list): return x*3

class A:
    @dispatch
    def encodes(self,x:str): return "h_"+x
    
assert len(A.encodes.methods)  == 1 # 🎉 Expected 1 b/c class redefinition

But you might have noticed that already...because now we never redefined dispatch 😎

In [ ]:
#|hide
#|eval: false
from nbdev import nbdev_export
nbdev_export()